In [1]:
tcl::tm::path add [file join . modules]
package require util

0.1


In [2]:
set data [read-input day08];

In [3]:
set width 25
set height 6
set layersize [expr {$width*$height}]
set pat [string repeat . $layersize]
set layers [regexp -all -inline $pat $data]

proc freq1s {d} {
  dict get $d 1
}

proc part1 {} {
  set min0freqs [lindex [lsort -integer -index 1 [lmap x $::layers {lsort -integer -stride 2 -index 0 [freq $x]}]] 0]
  return [expr {[dict get $min0freqs 1]*[dict get $min0freqs 2]}]
}

In [4]:
part1

2193


In [7]:
proc visualize-step {} {
  set layer 0
  set svgid [jupyter::html {<svg width="250" height="60"></svg>}]
  yield
  set svg [lrepeat 150 {}]
  foreach l [lreverse $::layers] {
    incr layer
    foreach y [range 0 5] {
       foreach x [range 0 24] {
       set idx [expr {$y*25+$x} ]
        set col [string index $l $idx]
        switch $col {
          0 {
              lset svg $idx {}
          }
          1 {
              lset svg $idx [svgpixel $x $y  0.01 red]
          }
          
        }
      }
    }
    jupyter::updatehtml $svgid "<b>layer: </b> $layer<br><svg viewbox=\"-1 -1 26 7\">[join $svg \n]</svg>"
    yield 1
  }
  yield 0
}


In [47]:
coroutine visualize visualize-step
while {[visualize]} {}

layer: 100

# Using JS
You can also directly modify the SVG using batched javascript commands

In [49]:
proc visualize-step-2 {} {
  set layer 0
  set svg [list {<svg viewbox="-1 -1 26 7">}]
  foreach y [range 0 5] {
    foreach x [range 0 24] {
        lappend svg [svgpixel $x $y 0.01 black]
    }
  }
  lappend svg </svg>
  set svg [join $svg \n]
  jupyter::html $svg
  jupyter::display application/javascript {
      function updateColor(id,color){
    var myrect = document.getElementById(id);
    myrect.style.fill = color;

    }
  }
  yield 
  foreach l [lreverse $::layers] {
    incr layer
    set js  [list {
      function updateColor(id,color){
    var myrect = document.getElementById(id);
    myrect.style.fill = color;

    } }]
    foreach y [range 0 5] {
       foreach x [range 0 24] {
       
 
       set idx [expr {$y*25+$x} ]
        set col [string index $l $idx]
        switch $col {
          0 {
              lappend js "updateColor('rect-$x-$y', 'white');"
          }
          1 {
              lappend js "updateColor('rect-$x-$y', 'black');"
          }
          
        }
      }
    }
    jupyter::display application/javascript [join $js \n]
    yield 1
  }
  yield 0
};

In [50]:
coroutine visualize-2 visualize-step-2
while {[visualize-2]} {}